In [1]:
##Import
import glob, os
from Bio import SeqIO
import pandas as pd
from collections import Counter
import fuzzysearch
import matplotlib as plt
import numpy as np
import itertools
plt.rcParams['pdf.fonttype'] = 42

In [86]:
samples = pd.read_excel("recode.xls")
display(samples)

,Phage,Gene,Plasmid,Direction,edit name 1,edit name 2,edit name 3,edit name 4,edit name 5,edit name 6,...,L_inside_10,R_inside_10,L_inside_11,R_inside_11,L_outside,R_outside,Rep1,Rep2,Rep3,Rep4
0,Lambda,L,pCF.88,R,NaN,NaN,NaN,NaN,T37C,C46T,...,NaN,NaN,NaN,NaN,TATGACCAGCCAACGTCCGA,ACTTTCGCAGTAAATCCCAT,msSBK-32-20,msSBK-32-31,msCF-2-07,msCF-2-20
1,Lambda,L,pCF.89,R,NaN,NaN,NaN,C28T,T37C,C46T,...,NaN,NaN,NaN,NaN,TATGACCAGCCAACGTCCGA,ACTTTCGCAGTAAATCCCAT,msSBK-32-21,msSBK-32-32,NaN,msCF-2-21
2,Lambda,L,pCF.90,R,NaN,NaN,C19T,C28T,T37C,C46T,...,NaN,NaN,NaN,NaN,TATGACCAGCCAACGTCCGA,ACTTTCGCAGTAAATCCCAT,msSBK-32-22,msSBK-32-33,msCF-2-09,msCF-2-22
3,Lambda,L,pCF.91,R,NaN,G10A,C19T,C28T,T37C,C46T,...,GNTTCCTTTC,ATTAANAAAC,NaN,NaN,TATGACCAGCCAACGTCCGA,ACTTTCGCAGTAAATCCCAT,msSBK-32-23,msSBK-32-34,msCF-2-10,msCF-2-23
4,Lambda,L,pCF.92,R,C4T,G10A,C19T,C28T,T37C,C46T,...,GNTTCCTTTC,ATTAANAAAC,TTTCNATTAA,AAACTTTCGC,TATGACCAGCCAACGTCCGA,ACTTTCGCAGTAAATCCCAT,msSBK-32-24,msSBK-32-35,NaN,msCF-2-24


In [17]:
df_reads_new = pd.read_excel("df_cleanedup.xlsx")
display(df_reads_new)

,Unnamed: 0,level_0,index,fastq,read,frequency,plasmid,replicate,edit nt 1,edit nt 2,edit nt 3,edit nt 4,edit nt 5,edit nt 6,edit nt 7,edit nt 8,edit nt 9,edit nt 10,edit nt 11
0,6224,6224,0,msSBK-32-20,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,6031,pCF.88,Rep1,NaN,NaN,NaN,NaN,T,C,C,NaN,NaN,NaN,NaN
1,6225,6225,0,msSBK-32-20,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,3633,pCF.88,Rep1,NaN,NaN,NaN,NaN,T,C,C,NaN,NaN,NaN,NaN
2,6230,6230,0,msSBK-32-20,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,61,pCF.88,Rep1,NaN,NaN,NaN,NaN,C,C,C,NaN,NaN,NaN,NaN
3,6232,6232,0,msSBK-32-20,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,71,pCF.88,Rep1,NaN,NaN,NaN,NaN,T,C,T,NaN,NaN,NaN,NaN
4,6233,6233,0,msSBK-32-20,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,1,pCF.88,Rep1,NaN,NaN,NaN,NaN,C,T,T,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38715,136406,136406,0,msCF-2-24,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,1,pCF.92,Rep4,C,G,C,C,T,C,C,C,C,C,C
38716,136407,136407,0,msCF-2-24,CAGCTAACGCGGATGAATATGACCGGCCAACGTCCGATATCACGAA...,1,pCF.92,Rep4,C,G,C,C,T,C,C,C,C,C,C
38717,136409,136409,0,msCF-2-24,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,1,pCF.92,Rep4,C,G,C,C,T,C,C,C,C,C,C
38718,136411,136411,0,msCF-2-24,GGCTGTCGCGGCTGAATATGACCAGCCAACGTCCGATATCACGAAG...,1,pCF.92,Rep4,C,G,C,C,T,C,C,C,C,C,C


In [78]:
edited_nucleotide_df.columns


Index(['WT nt 1 ', 'Ed nt 1', 'WT nt 2', 'Ed nt 2', 'WT nt 3', 'Ed nt 3',
       'WT nt 4', 'Ed nt 4', 'WT nt 5', 'Ed nt 5', 'WT nt 6', 'Ed nt 6',
       'WT nt 7', 'Ed nt 7', 'WT nt 8', 'Ed nt 8', 'WT nt 9', 'Ed nt 9',
       'WT nt 10', 'Ed nt 10', 'WT nt 11', 'Ed nt 11'],
      dtype='object')

In [87]:
edited_nucleotide_df = samples[[i for i in list(samples.columns) if (i[0:5]=='Ed nt' or i[0:5]=='WT nt')]].dropna()
edit_dict = dict()
for num in [str(i) for i in range(1, 12)]:
    edit_dict[f'edit nt {num}'] = dict()

    wt_allele = edited_nucleotide_df[f'WT nt {num}'].values[0]
    edited_allele = edited_nucleotide_df[f'Ed nt {num}'].values[0]
    
    edit_dict[f'edit nt {num}'][wt_allele] = 'w'
    edit_dict[f'edit nt {num}'][edited_allele] = 'e'
    
print(edit_dict)

{'edit nt 1': {'C': 'w', 'T': 'e'}, 'edit nt 2': {'G': 'w', 'A': 'e'}, 'edit nt 3': {'C': 'w', 'T': 'e'}, 'edit nt 4': {'C': 'w', 'T': 'e'}, 'edit nt 5': {'T': 'w', 'C': 'e'}, 'edit nt 6': {'C': 'w', 'T': 'e'}, 'edit nt 7': {'C': 'w', 'T': 'e'}, 'edit nt 8': {'C': 'w', 'T': 'e'}, 'edit nt 9': {'C': 'w', 'T': 'e'}, 'edit nt 10': {'C': 'w', 'T': 'e'}, 'edit nt 11': {'C': 'w', 'T': 'e'}}


In [69]:
edited_nucleotide_df

,WT nt 1,Ed nt 1,WT nt 2,Ed nt 2,WT nt 3,Ed nt 3,WT nt 4,Ed nt 4,WT nt 5,Ed nt 5,...,WT nt 7,Ed nt 7,WT nt 8,Ed nt 8,WT nt 9,Ed nt 9,WT nt 10,Ed nt 10,WT nt 11,Ed nt 11
4,C,T,G,A,C,T,C,T,T,C,...,C,T,C,T,C,T,C,T,C,T


In [132]:
print(edit_dict)

{'edit nt 1': {'C': 'w', 'T': 'e'}, 'edit nt 2': {'G': 'w', 'A': 'e'}, 'edit nt 3': {'C': 'w', 'T': 'e'}, 'edit nt 4': {'C': 'w', 'T': 'e'}, 'edit nt 5': {'T': 'w', 'C': 'e'}, 'edit nt 6': {'C': 'w', 'T': 'e'}, 'edit nt 7': {'C': 'w', 'T': 'e'}, 'edit nt 8': {'C': 'w', 'T': 'e'}, 'edit nt 9': {'C': 'w', 'T': 'e'}, 'edit nt 10': {'C': 'w', 'T': 'e'}, 'edit nt 11': {'C': 'w', 'T': 'e'}}


In [167]:
list_of_df_by_edit = [] 
for num in [str(i) for i in range(1, 12)]:
    top = df_reads_new.groupby(["plasmid", "replicate", f'edit nt {num}'])["frequency"].sum()
    bottom = df_reads_new.groupby(["plasmid", "replicate"])["frequency"].sum()
    df_by_edit = ((top/bottom)*100).dropna(axis=0).reset_index()
    
    # print(df_by_edit)
    # df_by_edit[f'edit nt {num}'].apply(lambda x: print(edit_dict[f'edit nt {num}'][x] if x in edit_dict[f'edit nt {num}'] else print('none')))
    df_by_edit['edit_or_not'] = df_by_edit[f'edit nt {num}'].apply(lambda x: edit_dict[f'edit nt {num}'][x] if x in edit_dict[f'edit nt {num}'] else np.nan)
    df_by_edit.dropna(axis=0, how='any', inplace=True)
    # print(df_by_edit.shape[0])
    df_by_edit['position'] = [f'edit nt {num}']*df_by_edit.shape[0]
    df_by_edit.rename(columns={f'edit nt {num}': 'edit nt'}, inplace=True)
    list_of_df_by_edit.append(df_by_edit)
    # print(df_by_edit)
    
big_df_by_edit = pd.concat(list_of_df_by_edit)
print(big_df_by_edit)

   plasmid replicate edit nt  frequency edit_or_not    position
1   pCF.92      Rep1       C  99.883359           w   edit nt 1
3   pCF.92      Rep1       T   0.066652           e   edit nt 1
5   pCF.92      Rep2       C  99.822809           w   edit nt 1
7   pCF.92      Rep2       T   0.150344           e   edit nt 1
9   pCF.92      Rep4       C  99.952756           w   edit nt 1
..     ...       ...     ...        ...         ...         ...
3   pCF.92      Rep1       T   0.011109           e  edit nt 11
5   pCF.92      Rep2       C  99.914089           w  edit nt 11
7   pCF.92      Rep2       T   0.021478           e  edit nt 11
9   pCF.92      Rep4       C  99.984252           w  edit nt 11
11  pCF.92      Rep4       T   0.009449           e  edit nt 11

[247 rows x 6 columns]


In [168]:
big_df_by_edit.to_excel("individual_recode_output.xlsx")